<a href="https://colab.research.google.com/github/tarvt/DeleSmell_SA_G5/blob/main/dataset_class/Blue-Block/blue_semantic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install docx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 54 kB 3.3 MB/s 
  Created wheel for docx: filename=docx-0.2.4-py3-none-any.whl size=53924 sha256=0471cdbacc96ff5fe10473c8d4f6b5aaf5eb25926885ccdc497bff7bb38250cf
  Stored in directory: /root/.cache/pip/wheels/78/ae/2e/5ce789557ca59281a463dabe9d6522e429be240322148e5379
Successfully built docx


In [ ]:
!pip install exceptions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement exceptions (from versions: none)
ERROR: No matching distribution found for exceptions


In [1]:
import pandas as pd
import os
import numpy as np
import re
from string import punctuation
import pandas as pd
import csv
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
save_file_csv = 'blue2.csv'
file_csv =  'blue.csv'
file_dir = '/content/drive/MyDrive/delesmell/dataset_class/Blue-Block' #Newly stored class address


In [4]:
fileList = []
def file_name(file_dir):
    for root, dirs, files in os.walk(file_dir):
#       print(root) #current directory path
#       print(files) #All non-directory subfiles in the current path
         for file in files:
            if file[-4:] == '.txt':
                fileList.append(os.path.join(root, file))

In [5]:
file_name(file_dir)

In [6]:
fileList

['/content/drive/MyDrive/delesmell/dataset_class/Blue-Block/Ground.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/Blue-Block/PowerUp.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/Blue-Block/Locator.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/Blue-Block/InfoWindow.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/Blue-Block/ResourceManager.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/Blue-Block/GroundType.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/Blue-Block/InfoPanel.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/Blue-Block/Menu.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/Blue-Block/Settings.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/Blue-Block/Main.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/Blue-Block/Options.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/Blue-Block/Game.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/Blue-Block/Human.txt']

In [14]:
class preprocessing():
  

  def lm_find_unchinese(self ,line):
    pattern = re.compile(r'[\u4e00-\u9fa5]')
    unchinese = re.sub(pattern,"",line) #Exclude Chinese characters
    unchinese = re.sub('[{}]'.format(punctuation),"",unchinese) #Exclude Chinese characters

    return unchinese
    
  def del_stopwords(self,line):
    dicts = {'\n': '','!': '', '"': '','：':'', '#': '', '$': '', '%': '', '&': '', "'": '', '(': '', ')': '', '*': '', '+': '', ',': '', '-': '', '.': '', '/': '', ':': '', ';': '', '<': '', '=': '', '>': '', '?': '', '@': '', '[': '', '\\': '', ']': '', '^': '', '_': '', '`': '', '{': '', '|': '', '}': '', '~': ''}
    punc_table = str.maketrans(dicts)
    new_line = line.translate(punc_table)

    return new_line

In [33]:
all_files_words = []
class_name = []
Brain_Class = []
for i, f in enumerate(fileList):
    with open(f,encoding='gb18030',errors='ignore') as file:
        filewords = []
        list2 = []
        is_brain = 0
        filename = os.path.basename(file.name) #example: Ground.txt
        name = filename[:-4] #example: Ground
        class_name.append(name)
        filewords.append(name) # file names
        list2.append(name)
        pre = preprocessing()
        for line in file.readlines():
            line = pre.lm_find_unchinese(line)
            line = pre.del_stopwords(line)
            values = line.split(' ')
            coefs = values[1:-1]
            coefs = [i for i in coefs if i != '']
            keyword_brain = 'hongshuai'            

            if  keyword_brain in line:
                is_brain = 1
            if len(coefs) != 0:
                if coefs != ' ':
                    if coefs != '\n':
                        for i in coefs:
                            filewords.append(i)

        all_files_words.append(filewords)
        
        if is_brain == 0:
            list2.append(False)
        else :
            list2.append(True)
        
        Brain_Class.append(list2)

In [20]:
Brain_Class = np.array(Brain_Class)
Brain_Class

array([['Ground', 'False'],
       ['PowerUp', 'True'],
       ['Locator', 'False'],
       ['InfoWindow', 'False'],
       ['ResourceManager', 'False'],
       ['GroundType', 'False'],
       ['InfoPanel', 'False'],
       ['Menu', 'False'],
       ['Settings', 'False'],
       ['Main', 'False'],
       ['Options', 'False'],
       ['Game', 'False'],
       ['Human', 'False']], dtype='<U15')

In [31]:
def additem(list):
    if len(list)<3:
        q = len(list)-1
        str = '_1'
        str2 = '_2'
        item = ''.join([list[q], str])
        item_2 = ''.join([list[q], str2])
        list.append(item)
        list.append(item_2)
    return list


In [34]:
news_df = np.array(all_files_words,dtype=object) 
print(news_df)#a list that contains files words in list (a list of lists) 

[list(['Ground', 'GroundType', 'return', 'int', 'return', 'int', 'return', 'GroundGame', 'gameint', 'xint', 'yGroundType', 'void', 'void', 'SetGroundTypeGroundType', 'if', 'Type', 'null'])
 list(['PowerUp', 'boolean', 'return', 'void', 'int', 'int', 'void', 'NewPositionint', 'xint', 'Ground', 'GroundType', 'if', 'grtIsWall', 'grtIsDeadly', 'gameLocatorGetHumanxy', 'null', 'int', 'int', 'if', 'grtIsPoison', 'PowerUpGame', 'void', 'TakeEffectHuman', 'SystemoutprintlnPowerUp', 'GetType', 'Human', 'human2', 'gameHumans'])
 list(['Locator', 'void', 'backgroundColorint', 'pos1int', 'Ground', 'if', 'groundGetType', 'GroundTypeFloor', 'void', 'checkKillsHuman', 'if', 'SettingsEnablePlayerKills', 'for', 'Human', 'other', 'gameHumans', 'if', 'other', 'human', 'otherGetX', 'humanGetX', 'otherGetY', 'humanGetY', 'void', 'checkPowerupsHuman', 'PowerUp', 'if', 'PU', 'null', 'Ground', 'GetGroundint', 'xint', 'if', 'x', '0', 'y', '0', 'x', 'gameWidth', 'y', 'gameHeight', 'return', 'return', 'Human', '

In [85]:
from keras import Input,Model
from keras.layers import Dense
class SemanticAnalysis():

  def calculate_acsm( file_words, n):
    vectorizer = CountVectorizer() #used to transform a given text into a vector on the basis of the frequency (count) of each word that occurs in the entire text.
    X = vectorizer.fit_transform(file_words) #for every word in file_words this function index then and give each word a uniq id 


    n_cap = 1
    svd_model = TruncatedSVD(n_components=n_cap)
    X2 = svd_model.fit_transform(X) # convert each word in to a number 

    D = svd_model.components_ #The right singular vectors of the input data.
    T = X2
    S = np.diagflat(svd_model.singular_values_) #one number

    X = np.dot(T, S)
    X = np.dot(X, D)
    c = np.dot(X, X.T)

    acsm = None
    sum = 0
    for i in c:
        for j in i:
            if j < 0:
                j = 0
            sum += j
        acsm = sum / n / n
    # print("-------------acsm----------------------")
    # print(acsm)

    return acsm

  def additem(list):
      if len(list) < 3:
          q = len(list) - 1
          str = '_1'
          str2 = '_2'
          item = ''.join([list[q], str])
          item_2 = ''.join([list[q], str2])
          list.append(item)
          list.append(item_2)

      return list
  def word2vec_train(index):
        list = []
        list.append(index[0])
        words = []

        additem(index)


        for word in index:
            if word not in words:
                words.append(word)


        word2int = {}
        for i,word in enumerate(words):
            word2int[word] = i
        print(word2int)



        WINDOW_SIZE=2
        data = []
        # for sentence in index:
        for idx,word in enumerate(index):
            for neighbor in index[max(idx - WINDOW_SIZE,0) : min(idx + WINDOW_SIZE,len(index))+ 1]:
                if neighbor != word:
                    data.append([word, neighbor])
    #                   print(data)
        df = pd.DataFrame(data,columns=['input','label'])



        ONE_HOT_DIM = len(words)
        #function to convert numbers to one hot vectors
        def to_one_hot_encoding(data_point_index):
            one_hot_encoding = np.zeros(ONE_HOT_DIM)
            one_hot_encoding[data_point_index] = 1
            return one_hot_encoding

        X = []
        Y = []

        for x, y in zip(df['input'],df['label']):
            X.append(to_one_hot_encoding(word2int[x]))
            Y.append(to_one_hot_encoding(word2int[y]))

        x_train = np.array(X)
        y_train = np.array(Y)


        #Defining the size of the embedding
        embed_size = 3
        xx = Input(shape=(x_train.shape[1],))
        yy = Dense(units=embed_size, activation='linear')(xx)
        yy =Dense(units=y_train.shape[1],activation='softmax')(yy)
        model = Model(inputs=xx, outputs=yy)
        model.compile(loss = 'categorical_crossentropy',optimizer = 'adam')
        #optimizing the network weights
        model.fit(x=x_train,y=y_train,
            batch_size=128,
            epochs=500
        )
        weights = model.get_weights()[0]


        for word in words:

            q = weights[df.get(word)]

            arr_new = np.sum(q, axis=1)
            arr_new = arr_new.tolist()

            list.append(arr_new)

            break
        return list

    

In [78]:
SemanticAnalysis = SemanticAnalysis()

In [49]:
q = 0
acsm= []
for file_words in all_files_words:
    lists = []
    for word in file_words:
        lists.append(word)

    if len(lists) > 1:
        acsm.append(SemanticAnalysis.calculate_acsm(lists,len(file_words)))
    else :
        acsm.append(1)


In [50]:
class_name

['Ground',
 'PowerUp',
 'Locator',
 'InfoWindow',
 'ResourceManager',
 'GroundType',
 'InfoPanel',
 'Menu',
 'Settings',
 'Main',
 'Options',
 'Game',
 'Human']

In [51]:
classname_acsm = []
for i in range(len(class_name)):
            t=[class_name[i],acsm[i]]
            classname_acsm.append(t)

classname_acsm = np.array(classname_acsm)

In [52]:
classname_acsm

array([['Ground', '0.09348862817814299'],
       ['PowerUp', '0.08164623123296835'],
       ['Locator', '0.024493187187538443'],
       ['InfoWindow', '0.010384221735473497'],
       ['ResourceManager', '0.02960662356547327'],
       ['GroundType', '0.6666666666666667'],
       ['InfoPanel', '0.061738947167331894'],
       ['Menu', '0.870134522310575'],
       ['Settings', '0.024649764151061015'],
       ['Main', '0.1674808368829604'],
       ['Options', '0.019540574269885418'],
       ['Game', '0.0799957515290302'],
       ['Human', '0.34293568715236417']], dtype='<U32')

In [53]:
df =pd.read_csv("/content/drive/MyDrive/delesmell/dataset_class/Blue-Block/blue.csv") 

In [58]:
df_name= df.iloc[:, 0]

In [59]:
Brain_Class

[['Ground', False],
 ['PowerUp', True],
 ['Locator', False],
 ['InfoWindow', False],
 ['ResourceManager', False],
 ['GroundType', False],
 ['InfoPanel', False],
 ['Menu', False],
 ['Settings', False],
 ['Main', False],
 ['Options', False],
 ['Game', False],
 ['Human', False]]

In [90]:
#add new column
ngList=[] #Prepare a list and store the data of the new column in it

for name in df_name:
#     print(index)
    q = 1
    for i in range(0,len(classname_acsm)):
        if name == classname_acsm[i][0]:
            ngList.append(classname_acsm[i][1])
            q = 0

    if q==1 :
        ngList.append(1)

In [91]:
ngList

['0.024493187187538443',
 '0.061738947167331894',
 1,
 '0.1674808368829604',
 '0.010384221735473497',
 '0.08164623123296835',
 '0.870134522310575',
 '0.019540574269885418',
 '0.34293568715236417',
 '0.09348862817814299',
 '0.6666666666666667',
 '0.0799957515290302',
 '0.02960662356547327',
 '0.024649764151061015']

In [73]:
brainList=[] #Prepare a list and store the data of the new column in it

for name in df_name:
#     print(index)
    q = 1
    for i in range(0,len(classname_acsm)):
        if name == (Brain_Class[i][0]):
            brainList.append(Brain_Class[i][1])
            q = 0
    if q==1 :
        brainList.append('0')
        

In [86]:
embedding = []
for item in all_files_words:
    embedding.append(SemanticAnalysis.word2vec_train(item))
print(embedding)

Streaming output truncated to the last 5000 lines.
2/2 [==============================] - 0s 5ms/step - loss: 3.7111
Epoch 4/500
2/2 [==============================] - 0s 5ms/step - loss: 3.7099
Epoch 5/500
2/2 [==============================] - 0s 4ms/step - loss: 3.7088
Epoch 6/500
2/2 [==============================] - 0s 4ms/step - loss: 3.7077
Epoch 7/500
2/2 [==============================] - 0s 4ms/step - loss: 3.7066
Epoch 8/500
2/2 [==============================] - 0s 4ms/step - loss: 3.7056
Epoch 9/500
2/2 [==============================] - 0s 5ms/step - loss: 3.7045
Epoch 10/500
2/2 [==============================] - 0s 4ms/step - loss: 3.7034
Epoch 11/500
2/2 [==============================] - 0s 5ms/step - loss: 3.7023
Epoch 12/500
2/2 [==============================] - 0s 7ms/step - loss: 3.7013
Epoch 13/500
2/2 [==============================] - 0s 2ms/step - loss: 3.7002
Epoch 14/500
2/2 [==============================] - 0s 4ms/step - loss: 3.6992
Epoch 15/500
2/2 [==

In [94]:
emblist=[]
for index in df_name:
#     print(index)
    q = 1
    for i in range(0,len(embedding)):
        if index == embedding[i][0]:
            # embedding[i][1].reshape((1,3))
            # l = embedding[i][1].tolist()
            emblist.append(embedding[i][1])
            q = 0
            # break
#             print('*')
    if q==1 :
        emblist.append([[1, 1, 1]])

In [95]:
emblist

[[[-12.12342643737793, 0.7957501411437988, -2.012115001678467]],
 [[-2.2341625690460205, 2.253973960876465, -0.6317392587661743]],
 [[1, 1, 1]],
 [[-7.780852317810059, -6.501304626464844, 7.163446426391602]],
 [[-2.450643301010132, 1.0330266952514648, 0.46852022409439087]],
 [[-2.6342906951904297, -0.05549773573875427, -0.5322811007499695]],
 [[1.6401408910751343, 0.003982275724411011, -1.5151407718658447]],
 [[-1.968711495399475, 2.728243827819824, 11.039443969726562]],
 [[-3.2038466930389404, 5.740797996520996, -2.7358155250549316]],
 [[0.28450918197631836, -0.1075981855392456, -0.5793519616127014]],
 [[-1.0044710636138916, -0.9385591745376587, 0.08188319206237793]],
 [[-2.4062137603759766, -4.984613418579102, -3.919346570968628]],
 [[-2.942981719970703, -2.1858389377593994, -1.359147310256958]],
 [[-0.30145904421806335, 2.9239797592163086, -1.4668363332748413]]]

In [96]:
word2vec = []
list1 = []
list2 = []
list3 = []
for i in range(0,len(emblist)):
    word2vec.append(emblist[i][0])

for i in range(0,len(word2vec)):
    list1.append(word2vec[i][0])
for i in range(0,len(word2vec)):
    list2.append(word2vec[i][1])
for i in range(0,len(word2vec)):
    list3.append(word2vec[i][2])


In [74]:
column=df.columns.tolist()

In [75]:
df['acsm'] = ngList 
df['is_brain'] = brainList 
df.to_csv(save_file_csv,mode = 'a',index =False)

In [76]:
df.head()

,NAME,AMW,ATDF,BOVM,BUR,CBO,CC,CM,CRIX,DAC,...,NOA,NOAM,NOD,NOM,NOPA,TCC,WMC,WOC,acsm,is_brain
0,Locator,4.78,18,0,1,7,3,4,2.5,1,...,1,0,0,9,0,0.57,43,1.00,0.024493187187538443,False
1,InfoPanel,3.33,5,0,1,3,1,1,0.0,0,...,5,0,0,3,0,1.00,10,1.00,0.061738947167331894,False
2,1,1.00,0,0,1,2,0,0,0.0,0,...,0,0,0,1,0,1.00,1,1.00,1,0
3,Main,1.94,6,0,1,5,6,9,0.0,2,...,7,0,0,17,3,0.11,33,0.83,0.1674808368829604,False
4,InfoWindow,1.67,8,0,1,5,1,5,0.0,4,...,6,0,0,3,1,0.00,5,0.67,0.010384221735473497,False


In [97]:
df['word2vec'] = list1
df['word2vec2'] = list2
df['word2vec3'] = list3

In [98]:
df.head()

,NAME,AMW,ATDF,BOVM,BUR,CBO,CC,CM,CRIX,DAC,...,NOM,NOPA,TCC,WMC,WOC,acsm,is_brain,word2vec,word2vec2,word2vec3
0,Locator,4.78,18,0,1,7,3,4,2.5,1,...,9,0,0.57,43,1.00,0.024493187187538443,False,-12.123426,0.795750,-2.012115
1,InfoPanel,3.33,5,0,1,3,1,1,0.0,0,...,3,0,1.00,10,1.00,0.061738947167331894,False,-2.234163,2.253974,-0.631739
2,1,1.00,0,0,1,2,0,0,0.0,0,...,1,0,1.00,1,1.00,1,0,1.000000,1.000000,1.000000
3,Main,1.94,6,0,1,5,6,9,0.0,2,...,17,3,0.11,33,0.83,0.1674808368829604,False,-7.780852,-6.501305,7.163446
4,InfoWindow,1.67,8,0,1,5,1,5,0.0,4,...,3,1,0.00,5,0.67,0.010384221735473497,False,-2.450643,1.033027,0.468520
